# NumPy

In [1]:
import numpy as np
from numpy import sin, pi

def plate_displacement_py(xx, yy, ww, a, b, P, xi, eta, D, max_i, max_j, max_m, max_n):
    for mm in range(1, max_m):
        for nn in range(1, max_n):
            for ii in range(max_i):
                for jj in range(max_j):
                    a_mn = 4 * P * sin(mm * pi * xi / a) * sin(nn * pi * eta / b) / (a * b)
                    ww[ii][jj] += (a_mn / (mm**2 / a**2 + nn**2 / b**2)**2
                                   * sin(mm * pi * xx[ii][jj] / a)
                                   * sin(nn * pi * yy[ii][jj] / b)
                                   / (pi**4 * D))

In [2]:
# Plate geometry
a = 1.0  # m
b = 1.0  # m
h = 50e-3  # m

# Material properties
E = 69e9  # Pa
nu = 0.35

# Series terms
max_m = 16
max_n = 16

# Computation points
# NOTE: With an odd number of points the center of the place is included in
# the grid
NUM_POINTS = 101
max_i = NUM_POINTS
max_j = NUM_POINTS

# Load
P = 10e3  # N
xi = a / 2
eta = a / 2

# Flexural rigidity
D = h**3 * E / (12 * (1 - nu**2))

# ---

# Set up domain
x = np.linspace(0, a, num=NUM_POINTS)
y = np.linspace(0, b, num=NUM_POINTS)
xx, yy = np.meshgrid(x, y)

In [3]:
# Compute displacement field
ww = np.zeros_like(xx)
plate_displacement_py(xx, yy, ww, a, b, P, xi, eta, D, max_i, max_j, max_m, max_n)

# Print maximum displacement
w_max = np.abs(ww).max()
print("Maximum displacement = %14.12f mm" % (w_max * 1e3))
print("alpha = %7.5f" % (w_max / (P * a**2 / D)))
print("alpha * P a^2 / D = %6.4f mm" % (0.01160 * P * a**2 / D * 1e3))

Maximum displacement = 0.141317840389 mm
alpha = 0.01158
alpha * P a^2 / D = 0.1416 mm


In [4]:
%timeit plate_displacement_py(xx, yy, ww, a, b, P, xi, eta, D, max_i, max_j, max_m, max_n)

1 loops, best of 3: 24.3 s per loop


## Vectorizando

Los bucles interiores sirven para ejecutar las operaciones matemáticas para cada uno de los puntos de la malla, pero si escribimos el código de esta manera estamos desaprovechando la potencia de NumPy: sus funciones matemáticas son más lentas para valores escalares que las contrapartidas del módulo `math`, y realmente la diferencia de rendimiento se aprecia cuando **vectorizamos** las operaciones.

In [5]:
def plate_displacement_np(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n):
    for mm in range(1, max_m):
        for nn in range(1, max_n):
            a_mn = 4 * P * sin(mm * pi * xi / a) * sin(nn * pi * eta / b) / (a * b)
            ww += (a_mn / (mm**2 / a**2 + nn**2 / b**2)**2
                   * sin(mm * pi * xx / a)
                   * sin(nn * pi * yy / b)
                   / (pi**4 * D))

In [6]:
# Compute displacement field
ww = np.zeros_like(xx)
plate_displacement_np(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n)

# Print maximum displacement
w_max = np.abs(ww).max()
print("Maximum displacement = %14.12f mm" % (w_max * 1e3))
print("alpha = %7.5f" % (w_max / (P * a**2 / D)))
print("alpha * P a^2 / D = %6.4f mm" % (0.01160 * P * a**2 / D * 1e3))

Maximum displacement = 0.141317840389 mm
alpha = 0.01158
alpha * P a^2 / D = 0.1416 mm


In [7]:
%timeit plate_displacement_np(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n)

1 loops, best of 3: 199 ms per loop
